In [10]:
# Part 1
import re
import itertools

data = open("input_12.txt").read().splitlines()
data = [x.split() for x in data]
# print(data)
# [print(line) for line in data]

total_combos = 0
# for each line in data:
for i,line in enumerate(data):
    key = [int(x) for x in line[1].split(',')]
    # Set up the line[0]ing for searching
    line[0] = line[0].join(['.','.']) # add '.' at the ends to help search line[0]ings later
    unknown = [i for i, letter in enumerate(line[0]) if letter == '?']
    broken = [x.start(0) for x in re.finditer(r'#',line[0])]
    # print(broken,unknown)
    test_combos = list(itertools.combinations(unknown, sum(key)-len(broken)))
    # print(test_combos)
    line[0] = re.sub(r'\?',r'.',line[0])  # fill in the ? with .
    # print(line[0],'key =',key,'broken =',broken,'unknown =',unknown)
    regexstr = ''
    for k in range(len(key)):
        regexstr = regexstr + r'\.+#{' + str(key[k]) + r'}'
    # print('regex = ',regexstr)

    # loop over the possible combinations of broken springs and validate
    for k,combo in enumerate(test_combos):
        temp = line[0]
        # temp[combo] = ''
        temp = [ch for ch in temp]
        for m in range(len(combo)):
            temp[combo[m]] = '#'
        temp = ''.join(temp)
        
        z = re.findall(regexstr,temp)
        if z!=[]:
            total_combos += 1
            # print(temp,z,'valid =',z!=[])

    # print(' ')
    # print('       end of line')
    # print(' ')

print('total combos = ',total_combos)


total combos =  6981


In [11]:
# Part 2 only works for the sample file
import re
import itertools

# count combos in 1 line
def combo_count(seq,key,regexstr):

    # Set up the line for searching
    seq = seq.join(['.','.']) # add '.' at the ends to help search line[0] later
    unknown = [i for i, letter in enumerate(seq) if letter == '?']
    broken = [x.start(0) for x in re.finditer(r'#',seq)]
    # print(broken,unknown)
    test_combos = list(itertools.combinations(unknown, sum(key)-len(broken)))
    # print(test_combos)
    seq = re.sub(r'\?',r'.',seq)  # fill in the ? with .

    # look thru the possible combinations and validate them
    combos = 0
    for k,combo in enumerate(test_combos):
        temp = seq
        temp = [ch for ch in temp]   # break into a list
        for m in range(len(combo)):
            temp[combo[m]] = '#'   # stack the #'s into the sequence
        temp = ''.join(temp)       # convert the list back into a string
        
        z = re.findall(regexstr,temp)
        if z!=[]:
            combos += 1
        # print(temp,z,'valid =',z!=[])
    return combos

def run_sequence(sequences,key,regexstr):
    if sequences[0][-1] == '#':
        sequences[1] = '.'+sequences[1][1:]
    if sequences[1][0] == '#':
        sequences[0] = sequences[0][:-2]+'.'
    if len(sequences) == 3:
        if sequences[1][-1] == '#':
            sequences[2] = '.'+sequences[2][1:]
        if sequences[2][0] == '#':
            sequences[1] = sequences[1][:-2]+'.'
    combos = [0]*len(sequences)
    for k,seq in enumerate(sequences):
        combos[k] = combo_count(seq,key,regexstr)
    if len(sequences) == 2:
        return combos[0] * combos[1]**4
    else:
        return combos[0] * combos[1]**3 * combos[2]

# main
data = open("input_12_test.txt").read().splitlines()
# data = open("input_12.txt").read().splitlines()
data = [x.split() for x in data]

# data = [data[x] for x in [0,2,3]]  # cut down the list for testing
# print(data)
# [print(line) for line in data]

total_combos = 0
# for each line in data:
for i,line in enumerate(data):
    if i % 100 == 0:
        print('line = ',i)
    key = [int(x) for x in line[1].split(',')]

    regexstr = ''
    for k in range(len(key)):
        regexstr = regexstr + r'\.+#{' + str(key[k]) + r'}'
        # print('regex = ',regexstr)
    # print(line[0], 'key =',key)


    # case: . on either end: add ? to each side and take higher of combos[0] * combos2**4

    # case: slide the divider over left until hit # and turn the first ? into .

    # case: slide the divider over right until hit # and turn the first ? into .
    # loop thru a sliding divider
    # combo_group = []
    # slide = 0
    # while line[0][-slide] != '#':
    start_q = re.findall(r'^\?+',line[0])
    end_q = re.findall(r'\?+$',line[0])
    sequence_combos = []

    # add the ? between and split at the ? (Try both and take the higher combo count)
    sequences = []                          # 12345? | 12345? | 12345? | 12345? | 12345
    sequences.append(line[0])               # 12345
    sequences.append('?'+line[0])           # ?12345
    more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    print('sequence combos =',sequence_combos)

    sequences = []                          # 12345? | 12345? | 12345? | 12345? | 12345
    sequences.append(line[0])               # 12345
    sequences.append(line[0]+'?')           # 12345?
    more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    print('sequence combos =',more_combos)

    sequences = []                       # ?2345?? | 2345?? | 2345?? | 2345?? | 2345
    sequences.append(line[0]+'??')       # ?2345??
    sequences.append(line[0][1:]+'??')   # 2345??
    sequences.append(line[0][1:])        # 2345
    # more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    # print('sequence combos =',more_combos)

    sequences = []                      # 1234 | ??1234 | ??1234 | ??1234 | ??12345
    sequences.append(line[0][:-1])      # 1234
    sequences.append('??'+line[0][:-1]) # ??234
    sequences.append('??'+line[0])      # ??12345
    # more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    # print('sequence combos =',more_combos)

    sequences = []                      # ??345??? | 345??? | 345??? | 345??? | 345
    sequences.append(line[0]+'???')     # ??345???
    sequences.append(line[0][2:]+'???') # 345???
    sequences.append(line[0][2:])       # 2345
    # more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    # print('sequence combos =',more_combos)

    sequences = []                       # 123 | ???123 | ???123 | ???123 | ???12345
    sequences.append(line[0][:-2])       # 123
    sequences.append('???'+line[0][:-2]) # ???23
    sequences.append('???'+line[0])      # 12345
    # more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    # print('sequence combos =',more_combos)

    sequences = []                       # 123? | ??123 | ???123 | ???123 | ???12345
    sequences.append(line[0][:-2])       # 123
    
    sequences.append('???'+line[0][:-2]) # ???23
    sequences.append('???'+line[0])      # 12345
    # more_combos = run_sequence(sequences,key,regexstr)
    sequence_combos.append(more_combos)
    # print('sequence combos =',more_combos)

    sequence_combos = max(sequence_combos)

    print('                               line combos =',sequence_combos)
    # print(' ')

    total_combos += sequence_combos

print('total combos = ',total_combos)

# 1,873,485,728,309 is too low
# 2,717,079,803,569 is not right 1m22s


line =  0
sequence combos = [1]
sequence combos = 0
                               line combos = 1
sequence combos = [16384]
sequence combos = 1024
                               line combos = 16384
sequence combos = [1]
sequence combos = 1
                               line combos = 1
sequence combos = [16]
sequence combos = 1
                               line combos = 16
sequence combos = [2500]
sequence combos = 1024
                               line combos = 2500
sequence combos = [100000]
sequence combos = 506250
                               line combos = 506250
total combos =  525152


In [130]:
# Part 1 with recursion from HyperNutrino https://www.youtube.com/watch?v=g3Ms5e7Jdqo
def count(cfg, nums):
    if cfg == '':
        return 1 if nums == () else 0
    if nums == ():
        return 0 if '#' in cfg else 1
    
    result = 0
    if cfg[0] in '.?':
        result += count(cfg[1:], nums)
    if cfg[0] in '#?':
        if nums[0] <= len(cfg) and '.' not in cfg[:nums[0]] and (nums[0] == len(cfg) or cfg[nums[0]] != '#'):
            result += count(cfg[nums[0]+1:], nums[1:])
    return result


# main
total = 0
lines = open("input_12.txt").read().splitlines()
for line in lines:
    cfg,nums = line.split()
    nums = tuple(map(int, nums.split(',')))
    # print(cfg, nums)
    line_count = count(cfg, nums)
    # print('                           line count =',line_count)
    total += line_count
print(total)



6981


In [1]:
# Part 2 with recursion from HyperNutrino https://www.youtube.com/watch?v=g3Ms5e7Jdqo
def count(cfg, nums):
    if cfg == '':
        return 1 if nums == () else 0
    if nums == ():
        return 0 if '#' in cfg else 1
    
    key = (cfg, nums)
    if key in cache:
        return cache[key]
    
    result = 0
    if cfg[0] in '.?':
        result += count(cfg[1:], nums)
    if cfg[0] in '#?':
        if nums[0] <= len(cfg) and '.' not in cfg[:nums[0]] and (nums[0] == len(cfg) or cfg[nums[0]] != '#'):
            result += count(cfg[nums[0]+1:], nums[1:])

    cache[key] = result
    return result


# main
cache = {}  # build dictionary with memoization
total = 0
lines = open("input_12.txt").read().splitlines()
for i,line in enumerate(lines):
    # print(i) if i % 10 == 0 else 0
    cfg,nums = line.split()
    cfg = '?'.join([cfg]*5)
    nums = tuple(map(int, 5*nums.split(',')))
    # print(cfg, nums)
    line_count = count(cfg, nums)
    # print('                           line count =',line_count)
    total += line_count
print(total)



4546215031609
